In [9]:
'''This project aims to build the best elo rating systems for teams in the NBA. Below, I've provided outlines for the
two methods you will need to modify: win_prob() and change_elo(). I will use these two methods of yours to test the
effectiveness of your elo ranking system. I've attached the schedules and outcomes of five historical NBA seasons for
you to train your data (from SR) and will be using the most recently completely season to test. Please do not pull
the testing data as that will defeat the purpose of this project. Keep in mind, this is the bare minimum outline. 
You are encouraged to pull this code and write any tuning algorithms you deem necessary to further build your elo
system. Best of luck!'''

"This project aims to build the best elo rating systems for teams in the NBA. Below, I've provided outlines for the\ntwo methods you will need to modify: win_prob() and change_elo(). I will use these two methods of yours to test the\neffectiveness of your elo ranking system. I've attached the schedules and outcomes of five historical NBA seasons for\nyou to train your data (from SR) and will be using the most recently completely season to test. Please do not pull\nthe testing data as that will defeat the purpose of this project. Keep in mind, this is the bare minimum outline. \nYou are encouraged to pull this code and write any tuning algorithms you deem necessary to further build your elo\nsystem. Best of luck!"

In [10]:
import pandas as pd
import numpy as np
import sklearn as sl
import math

In [11]:
#read in data

games = pd.read_csv('NBA_2020_21.csv')

In [12]:
#initial elo for each team

teams = {
    "Brooklyn Nets":          0, #set initial values here (float)
    "Milwaukee Bucks":        0,
    "Golden State Warriors":  0,
    "Los Angeles Lakers":     0,
    "Indiana Pacers":         0,
    "Charlotte Hornets":      0,
    "Chicago Bulls":          0,
    "Detroit Pistons":        0,
    "Boston Celtics":         0,
    "New York Knicks":        0,
    "Washington Wizards":     0,
    "Toronto Raptors":        0,
    "Cleveland Cavaliers":    0,
    "Memphis Grizzlies":      0,
    "Houston Rockets":        0,
    "Minnesota Timberwolves": 0,
    "Philadelphia 76ers":     0,
    "New Orleans Pelicans":   0,
    "Orlando Magic":          0,
    "San Antonio Spurs":      0,
    "Oklahoma City Thunder":  0,
    "Utah Jazz":              0,
    "Sacramento Kings":       0,
    "Portland Trail Blazers": 0,
    "Denver Nuggets":         0,
    "Phoenix Suns":           0,
    "Dallas Mavericks":       0,
    "Atlanta Hawks":          0,
    "Miami Heat":             0,
    "Los Angeles Clippers":   0
}

In [13]:
#given two teams and their elos, return the probability of winning for the first team
#input: team 1 elo (float), team 2 elo (float)
#output: team 1 win probability between 0 and 1 (float)
awayFactor = 0.9

def win_prob(team_1_elo, team_2_elo) -> float:
    eloDiff = team_2_elo - team_1_elo # this ordering gives win probability of team 1
    team_1_win_prob = awayFactor * (10**(eloDiff / 420) + 1)**-1
    
    return team_1_win_prob

In [14]:
#given two teams and their original elos, return their new elos considering the outcome of their game
#input: team 1 elo (float), team 2 elo (float), game (DataFrame)
#ouput: new team 1 elo (float), new team 2 elo (float)

def change_elo(team_1_elo, team_2_elo, game) -> float:
    #K-factor is how much a win/loss affects the ELO
    #S1 is indicator variable for outcome of the game, 1 for team 1 win, 0 for team 1 loss, S2 is for team 2

    pointDiff = game['team_1_score'] - game['team_2_score']

    K = 24
    if game['team_1_score'] > game['team_2_score']:
        S1 = 1
        S2 = 0
    else:
        S1 = 0
        S2 = 1

    new_team_1_elo = team_1_elo + K * (S1 - win_prob(team_1_elo, team_2_elo)) + pointDiff/4
    new_team_2_elo = team_2_elo + K * (S2 - (1 - win_prob(team_1_elo, team_2_elo))) - pointDiff/4
    
    return new_team_1_elo, new_team_2_elo

In [15]:
#how predictive is the elo system?
#squared loss forcasting test (Brier Score)
#the lower the score, the better
def test() -> float:
    score = 0
    gCount = 0
    for index, row in games.iterrows():
        gCount = gCount + 1
        team_1, team_2 = row['team_1'], row['team_2']
        current_team_1_elo, current_team_2_elo = teams.get(team_1), teams.get(team_2)
        
        win = win_prob(current_team_1_elo, current_team_2_elo)
        
        new_team_1_elo, new_team_2_elo = change_elo(current_team_1_elo, current_team_2_elo, row)
        teams.update({team_1: new_team_1_elo})
        teams.update({team_2: new_team_2_elo})
        
        if row['team_1_score'] > row['team_2_score']:
            score = score + 2*(1 - win)**2
        else:
            score = score + 2*win**2
            
    return score/gCount

test()            

0.4516051947048719

In [16]:
def print_elo_values(teams):
    # Sort the teams by their ELO values in descending order
    sorted_teams = sorted(teams.items(), key=lambda x: x[1], reverse=True)

    # Print the ELO values of each team in a formatted way
    print("{:<25s} {:<10s}".format("Team", "ELO Rating"))
    print("-" * 35)
    for team, elo in sorted_teams:
        print("{:<25s} {:<10.2f}".format(team, elo))

print_elo_values(teams)


Team                      ELO Rating
-----------------------------------
Milwaukee Bucks           286.84    
Phoenix Suns              265.86    
Los Angeles Clippers      202.72    
Utah Jazz                 200.93    
Philadelphia 76ers        195.07    
Brooklyn Nets             186.14    
Atlanta Hawks             151.12    
Denver Nuggets            140.41    
Portland Trail Blazers    102.55    
Dallas Mavericks          88.46     
New York Knicks           83.12     
Los Angeles Lakers        62.12     
Memphis Grizzlies         53.70     
Golden State Warriors     49.62     
Miami Heat                33.78     
Washington Wizards        0.15      
Boston Celtics            -21.59    
Indiana Pacers            -35.54    
Chicago Bulls             -39.69    
New Orleans Pelicans      -55.56    
San Antonio Spurs         -70.32    
Sacramento Kings          -80.69    
Charlotte Hornets         -101.46   
Minnesota Timberwolves    -125.74   
Toronto Raptors           -141.54   
De